<a href="https://colab.research.google.com/github/gmmanzo/Seizure-Detection/blob/main/Seizure_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a seizure detection model using Time Series forecasting with the folllowing dataset https://physionet.org/content/chbmit/1.0.0/chb01/#files-panel

### To do this, we will first need to Extract and Visualize the Data to ensure the efficacy and reliability of the data,

Check for Recall(Main Part), Precision, F1 Score, & MAE in all of the model:

1. Naive Model, (Window = 7, Horizon = 1)
2. Basic Sequential Model (Window = 7, Horizon = 1)
3. Basic CNN (Window = 7, Horizon = 1)
4. Basic RNN (LSTM) (Window = 7, Horizon = 1)
5. Absolutely insane: Hybrid Model https://arxiv.org/html/2501.10342v1 (99.83% accuracy)

# Extract the data

In [2]:
!wget "https://physionet.org/content/siena-scalp-eeg/1.0.0/PN00/PN00-1.edf"
f= open("PN00-1.edf", "r", encoding = "utf-8")

--2025-01-30 23:19:33--  https://physionet.org/content/siena-scalp-eeg/1.0.0/PN00/PN00-1.edf
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5104 (5.0K) [text/html]
Saving to: ‘PN00-1.edf’

PN00-1.edf          100%[===================>]   4.98K  --.-KB/s    in 0s      

2025-01-30 23:19:33 (1.57 GB/s) - ‘PN00-1.edf’ saved [5104/5104]



In [3]:
pip install pyedflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyedflib: filename=pyEDFlib-0.1.39-cp311-cp311-linux_x86_64.whl size=2624432 sha256=539132974cfcdf87ca8725e65ddaec126130648a5ac062f1d050b4b64e487664
  Stored in directory: /root/.cache/pip/wheels/6d/25/41/b9605de2c5973791e87828347971b06712b12a14add875bff6
Successfully built pyedflib


In [1]:
# Load all data
import pyedflib # Used for edf files
import pandas as pd

# Load the EDF file
file_path = "/content/PN00-1.edf"
edf = pyedflib.EdfReader(file_path)

# Extract signal labels
signal_labels = edf.getSignalLabels()

# Extract signal data
signals = []
for i in range(edf.signals_in_file):
    signals.append(edf.readSignal(i))

# Create a DataFrame
df = pd.DataFrame(signals).transpose()
df.columns = signal_labels

# Close the EDF file
edf.close()

# Display the DataFrame
print(df)




ModuleNotFoundError: No module named 'pyedflib'

In [5]:
# Omit all columns except for EEG Fp1 and EEG Fp2

# Load the EDF file
import numpy as np

file_path = '/content/PN00-1.edf'
edf = pyedflib.EdfReader(file_path)

# Extract signal labels
signal_labels = edf.getSignalLabels()
print("Available signals:", signal_labels)  # Debug

# Find indices of 'EEG Fp1' and 'EEG Fp2'
desired_labels = ['EEG Fp1', 'EEG Fp2']
indices = [i for i, label in enumerate(signal_labels) if label in desired_labels]

# Debug: Check indices found
print(f"EEG Fp1 Index: {indices[0] if len(indices) > 0 else 'Not Found'}")
print(f"EEG Fp2 Index: {indices[1] if len(indices) > 1 else 'Not Found'}")

# Ensure both channels exist
if len(indices) < 2:
    raise ValueError("EEG Fp1 or EEG Fp2 not found in the EDF file!")

# Extract only the required signals
signals_new = [edf.readSignal(i).flatten() for i in indices]

# Get sample rates
fs_fp1 = edf.getSampleFrequency(indices[0])
fs_fp2 = edf.getSampleFrequency(indices[1])

print(f"Sample Rate of EEG Fp1: {fs_fp1} Hz")
print(f"Sample Rate of EEG Fp2: {fs_fp2} Hz")

# If sample rates are different, print a warning
if fs_fp1 != fs_fp2:
    print("⚠️ Warning: EEG Fp1 and EEG Fp2 have different sample rates!")

#Generate time array based on first signal length
n_samples = len(signals_new[0])
time = np.arange(n_samples)/fs_fp1

# Create a DataFrame
df_new = pd.DataFrame(np.column_stack([time, signals_new[0], signals_new[1]]),
                      columns=['Time'] + desired_labels)
edf.close()

# Create a DataFrame without time array
#df_new = pd.DataFrame(signals_new).transpose()
#df_new.columns = desired_labels  # Keeping only EEG Fp1 and EEG Fp2

# Display the DataFrame
print(df_new)

OSError: /content/PN00-1.edf: the file is not EDF(+) or BDF(+) compliant (it contains format errors)

In [ ]:
# Visualize data
import matplotlib.pyplot as plt
plt.figure(figsize = (12,6))
plt.plot(df_new['Time'], df_new['EEG Fp1'], label='EEG Fp1', color='blue')
plt.plot(df_new['Time'], df_new['EEG Fp2'], label='EEG Fp2', color='red')

plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('EEG Signals')
plt.legend()
plt.grid()
plt.show()

## Feature extraction and Seizure classification:

1. Understand EEG format, look at data frame using "name.head()"
2. Choose machine learning task
* EEG classification: detecting seizures
* EEG regression: predicting cognitive load
* EEG time series analysis: forecasting (LSTM, CNNs)
3. Feature engineering to extract features like:
* Statistical Features: Mean, variance, skewness, kurtosis
* Frequency Features: Power spectral density (PSD), Fast Fourier Transform (FFT)
* Wavelet Features: Decomposition of EEG signals
* Time-Series Features: Moving averages, peak detection
4. Format for machine learning
5. Train ML model
6. Evaluate and optimize

**Sliding windows**: split EEG data into smaller structured segments that overlap based on step size to ensure smooth transitions. Different window sizes can help capture short term or long term patterns

In [ ]:
#Load data
import pyedflib
import numpy as np
import pandas as pd

#Load edf file
file_path = '/content/PN00-1.edf'
edf = pyedflib.EdfReader(file_path)

#Extract signal labels and check
EEG_labels = edf.getSignalLabels()
print("EEG labels:", EEG_labels)

In [ ]:
# Segment Data into Windows
# This is useful when there is a lot of data
window_size = 512 #number of samples per second
step_size = 50 #overlap between windows

# Create sliding windows
def create_windows(eeg_data, window_size, step_size):
    windows = [eeg_data[i:i+window_size] for i in range(0, len(eeg_data) - window_size + 1, step_size)]
    return np.array(windows)

# Apply sliding window to each channel signal
sliding_windows = {}
for i, label in enumerate(EEG_labels):
    channel_signal = signals[i]
    sliding_windows[label] = create_windows(channel_signal, window_size, step_size)

In [ ]:
from scipy.stats import skew, kurtosis
from scipy.signal import welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Define feature extraction function (Import this into our github helper functions)
def extract_features(windows):
    features = {}

    # Time-domain features
    features['mean'] = np.mean(windows)
    features['std'] = np.std(windows)
    features['skew'] = skew(windows)
    features['kurtosis'] = kurtosis(windows)

    # Frequency-domain features (Power Spectral Density)
    f, Pxx = welch(windows, fs=512, nperseg=512)
    features['power_band'] = np.sum(Pxx)  # Total power

    return features

# Extract features for each window in each channel
features = []
for label, windows in sliding_windows.items():
    for window in windows:
        features.append(extract_features(windows))

# Convert feature list to a DataFrame
df_features = pd.DataFrame(features)

# Display the first few rows of the extracted features
print(df_features.head())

